In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_mdq_store_dc_sku_daily
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
) 
AS
WITH
main AS (
    SELECT      
        a.business_unit,
        a.store_id,
        a.store_name,
        a.business_model,
        a.mien,
        a.city,
        a.mch2_id,
        a2.mch3_id,
        a2.mch4_id,
        a.product_id,
        a.product_name,
        a.manufacturer_id,
        a.manufacturer_name,
        a1.mdq,
        a1.external_report_mdq,
        a1.internal_report_mdq,
        a1.old_mdq,
        a1.mdq_age,
        a1.mdq_available,
        a.dc_id,
        a.dc_name,
        a.replenishment_mode,
        a.XD_DC_ID,
        IFNULL(a1.sku_type, 'FRESH') sku_type,
        a.assortment_type
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_daily a1 USING (store_id, product_id)
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product a2 USING (product_id)
    UNION ALL 
    SELECT      
        a.business_unit,
        a.store_id,
        a.store_name,
        a.business_model,
        a.mien,
        a.city,
        a.mch2_id,
        a2.mch3_id,
        a2.mch4_id,
        a.product_id,
        a.product_name,
        a.manufacturer_id,
        a.manufacturer_name,
        a1.mdq,
        a1.external_report_mdq,
        a1.internal_report_mdq,
        a1.old_mdq,
        a1.mdq_age,
        a1.mdq_available,
        NULL dc_id,
        NULL dc_name,
        'DSD' replenishment_mode,
        NULL XD_DC_ID,
        IFNULL(a1.sku_type, 'FRESH') sku_type,
        a.assortment_type
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dsd a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_daily a1 USING (store_id, product_id)
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product a2 USING (product_id)
    UNION ALL
    SELECT      
        a.business_unit,
        a.store_id,
        a.store_name,
        CASE
            WHEN a.business_unit = '1500' THEN 'Supermarket'
            WHEN a.business_unit = '2000' AND a.concept IN ('WiN','Rural') THEN a.concept
            ELSE 'Urban'
        END business_model,
        a.mien,
        a.city,
        a2.mch2_id,
        a2.mch3_id,
        a2.mch4_id,
        a2.product_id,
        a2.product_name,
        a2.manufacturer_id,
        a2.manufacturer_name,
        a1.mdq,
        a1.external_report_mdq,
        a1.internal_report_mdq,
        a1.old_mdq,
        a1.mdq_age,
        a1.mdq_available,
        NULL dc_id,
        NULL dc_name,
        NULL replenishment_mode,
        NULL XD_DC_ID,
        a1.sku_type,
        NULL assortment_type
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_daily a1 
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store a USING (store_id)
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product a2 USING (product_id)
    WHERE a1.mdq IS NOT NULL 
        AND a1.sku_type = 'FRESH'
)
SELECT      
    a.*,
    IF(a1.sku IS NOT NULL, 'YES','NO') is_problem_article
FROM main a
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store a2 USING (store_id)
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_problem_article a1 ON a.business_unit = a1.business_unit 
    AND a2.region_vn = a1.area
    AND a.product_id = a1.sku
;
""")